In [1]:
! pip install ray[client]
! pip install transformers
! pip install accelerate
! pip install evaluate
! pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os

import evaluate
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

import ray.train.huggingface.transformers
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer
import ray
import torch

ray.init(address='ray://thalis.madgik.di.uoa.gr:10001')

/home/nikos/workspace/ray-docker-compose/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-11 19:50:45,334	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-01-11 19:50:45,427	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-01-11 19:50:45,447	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.8.10
Ray version:,2.9.0
Dashboard:,http://88.197.52.51:8265


In [5]:
# [1] Encapsulate data preprocessing, training, and evaluation
# logic in a training function
# ============================================================
def train_func(config):
    # Datasets
    dataset = load_dataset("yelp_review_full")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    small_train_dataset = (
        dataset["train"].select(range(1000)).map(tokenize_function, batched=True)
    )
    small_eval_dataset = (
        dataset["test"].select(range(100)).map(tokenize_function, batched=True)
    )

    # Model
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=5, device_map={'':'cpu'}, torch_dtype=torch.float16
    )

    # Evaluation Metrics
    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # Hugging Face Trainer
    training_args = TrainingArguments(
        output_dir="test_trainer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to="none",
        per_device_train_batch_size=4
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )

    # [2] Report Metrics and Checkpoints to Ray Trainentrypoint
    # ===============================================
    callback = ray.train.huggingface.transformers.RayTrainReportCallback()
    trainer.add_callback(callback)

    # [3] Prepare Transformers Trainer
    # ================================
    trainer = ray.train.huggingface.transformers.prepare_trainer(trainer)

    # Start Training
    trainer.train()


# [4] Define a Ray TorchTrainer to launch `train_func` on all workers
# ===================================================================
ray_trainer = TorchTrainer(
    train_func,
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False),
    # [4a] If running in a multi-node cluster, this is where you
    # should configure the run's persistent storage that is accessible
    # across all worker nodes.
    run_config=ray.train.RunConfig(storage_path="/ray"),
)
result: ray.train.Result = ray_trainer.fit()

# [5] Load the trained model.
with result.checkpoint.as_directory() as checkpoint_dir:
    checkpoint_path = os.path.join(
        checkpoint_dir,
        ray.train.huggingface.transformers.RayTrainReportCallback.CHECKPOINT_NAME,
    )
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

(TunerInternal pid=12709) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
(TunerInternal pid=12709) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=12709) 
(TunerInternal pid=12709) View detailed results here: /ray/TorchTrainer_2024-01-11_19-52-59
(TunerInternal pid=12709) To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/TorchTrainer_2024-01-11_19-52-59`


(TorchTrainer pid=990, ip=127.0.0.1) The actor died unexpectedly before finishing this task.
(TorchTrainer pid=990, ip=127.0.0.1) 	class_name: 
(TorchTrainer pid=990, ip=127.0.0.1) 	actor_id: 87a6186050c65023aaa098b408000000
(TorchTrainer pid=990, ip=127.0.0.1) Traceback (most recent call last):
(TorchTrainer pid=990, ip=127.0.0.1)   File "/usr/local/lib/python3.8/dist-packages/ray/train/_internal/backend_executor.py", line 190, in start
(TorchTrainer pid=990, ip=127.0.0.1)     self._backend.on_start(self.worker_group, self._backend_config)
(TorchTrainer pid=990, ip=127.0.0.1)   File "/usr/local/lib/python3.8/dist-packages/ray/train/torch/config.py", line 198, in on_start
(TorchTrainer pid=990, ip=127.0.0.1)     ray.get(setup_futures)
(TorchTrainer pid=990, ip=127.0.0.1)   File "/usr/local/lib/python3.8/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
(TorchTrainer pid=990, ip=127.0.0.1)     return fn(*args, **kwargs)
(TorchTrainer pid=990, ip=127.0.0.1)   F

(TunerInternal pid=11894) 
(TunerInternal pid=11894) Training errored after 0 iterations at 2024-01-11 19:53:02. Total running time: 1min 48s
(TunerInternal pid=11894) Error file: /root/ray_results/TorchTrainer_2024-01-11_19-51-12/TorchTrainer_033e7_00000_0_2024-01-11_19-51-14/error.txt
(TunerInternal pid=11894) 


(TrainTrainable pid=323, ip=127.0.0.1) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(TorchTrainer pid=323, ip=127.0.0.1) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.


...

(TunerInternal pid=12709) Trial task failed for trial TorchTrainer_42b96_00000
(TunerInternal pid=12709) Traceback (most recent call last):
(TunerInternal pid=12709)   File "/usr/local/lib/python3.8/dist-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
(TunerInternal pid=12709)     result = ray.get(future)
(TunerInternal pid=12709)   File "/usr/local/lib/python3.8/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
(TunerInternal pid=12709)     return fn(*args, **kwargs)
(TunerInternal pid=12709)   File "/usr/local/lib/python3.8/dist-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
(TunerInternal pid=12709)     return func(*args, **kwargs)
(TunerInternal pid=12709)   File "/usr/local/lib/python3.8/dist-packages/ray/_private/worker.py", line 2624, in get
(TunerInternal pid=12709)     raise value.as_instanceof_cause()
(TunerInternal pid=12709) ray.exceptions.RayTaskError(RayActorError): ray::TrainTrainable.train(

TrainingFailedError: The Ray Train run failed. Please inspect the previous error messages for a cause. After fixing the issue (assuming that the error is not caused by your own application logic, but rather an error such as OOM), you can restart the run from scratch or continue this run.
To continue this run, you can use: `trainer = TorchTrainer.restore("/home/nikos/ray_results/TorchTrainer_2024-01-11_19-52-55")`.
To start a new run that will retry on training failures, set `train.RunConfig(failure_config=train.FailureConfig(max_failures))` in the Trainer's `run_config` with `max_failures > 0`, or `max_failures = -1` for unlimited retries.

(TunerInternal pid=12709) Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
(TunerInternal pid=12709) Trials did not complete: [TorchTrainer_42b96_00000]


(TunerInternal pid=12709) 


Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2024-01-11 19:53:38,627	WARNING dataclient.py:403 -- Encountered connection issues in the data channel. Attempting to reconnect.
2024-01-11 19:54:08,836	WARNING dataclient.py:410 -- Failed to reconnect the data channel
